In [2]:
from random import random
from time import process_time_ns

from TsMon import TsMon
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt


def euclidean_distance(seq1, seq2):
    # 确保序列长度相同
    if len(seq1) != len(seq2):
        raise ValueError("序列长度必须相同")

    # 转换为 numpy 数组并计算欧氏距离
    seq1 = np.array(seq1)
    seq2 = np.array(seq2)
    return np.sqrt(np.sum((seq1 - seq2) ** 2))


def cosine_similarity(seq1, seq2):

    # 转换为 numpy 数组
    seq1 = np.array(seq1, dtype=np.int64)
    seq2 = np.array(seq2, dtype=np.int64)

    max_value = 0
    for i in range(-5, 6):
        if i < 0:
            i_seq1 = np.concatenate((np.zeros(abs(i)), seq1))
        else:
            i_seq1 = seq1[i:]

        min_len = min(len(i_seq1), len(seq2))
        i_seq1 = i_seq1[:min_len]
        seq2 = seq2[:min_len]

        # 计算点积
        dot_product = np.dot(i_seq1, seq2)

        # 计算模长
        norm1 = np.sqrt(np.sum(i_seq1 ** 2))
        norm2 = np.sqrt(np.sum(seq2 ** 2))

        # 避免除以零
        if norm1 == 0 or norm2 == 0:
            return 0.0

        max_value = max(max_value, dot_product / (norm1 * norm2))

    # 计算余弦相似度
    return max_value


def average_relative_error(seq1, seq2):
    seq1 = np.array(seq1, dtype=np.float64)
    seq2 = np.array(seq2, dtype=np.float64)

    min_value = 0x7fffffff
    for i in range(-5, 6):
        if i < 0:
            i_seq1 = np.concatenate((np.zeros(abs(i)), seq1))
        else:
            i_seq1 = seq1[i:]

        min_len = min(len(i_seq1), len(seq2))
        i_seq1 = i_seq1[:min_len]
        seq2 = seq2[:min_len]

        # 计算绝对误差
        abs_error = np.abs(i_seq1 - seq2)

        # 计算相对误差（避免除以零）
        relative_error = np.divide(abs_error, np.abs(i_seq1), out=np.zeros_like(abs_error), where=i_seq1 != 0)

        min_value = min(min_value, np.mean(relative_error))

    # 计算平均相对误差
    return min_value


def energy_similarity(f, f_hat):

    # 转换为 numpy 数组并确保浮点数类型
    seq1 = np.array(f, dtype=np.float64)
    seq2 = np.array(f_hat, dtype=np.float64)

    max_value = 0
    for i in range(-5, 6):
        if i < 0:
            i_seq1 = np.concatenate((np.zeros(abs(i)), seq1))
        else:
            i_seq1 = seq1[i:]

        min_len = min(len(i_seq1), len(seq2))
        i_seq1 = i_seq1[:min_len]
        seq2 = seq2[:min_len]

        # 计算能量（平方和的平方根）
        energy_f = np.sqrt(np.sum(i_seq1 ** 2))
        energy_f_hat = np.sqrt(np.sum(seq2 ** 2))

        # 避免除以零
        if energy_f == 0 or energy_f_hat == 0:
            return 0.0

        # 根据公式计算相似度
        if energy_f <= energy_f_hat:
            max_value = max(max_value, energy_f / energy_f_hat)
        else:
            max_value = max(max_value, energy_f_hat / energy_f)

    return max_value


def count_packets_by_key(packets, window_size, target_key):
    if not packets:
        return []

    # 获取所有时间戳以确定范围
    all_timestamps = []
    for key, bits, timestamp, _ in packets:
        if key == target_key:
            all_timestamps.append(timestamp)
    if not all_timestamps:
        return []

    min_time = min(all_timestamps)
    max_time = max(all_timestamps)

    # 计算窗口数量
    num_windows = int((max_time - min_time) // window_size) + 1

    # 初始化计数数组
    counters = [0] * num_windows

    # 统计 key=2 的数据包
    for key, bits, time, _ in packets:
        if key == target_key:
            # 计算窗口索引
            window_index = int((time - min_time) // window_size)
            if 0 <= window_index < num_windows:
                counters[window_index] += bits

    return counters


def count_all_packets_by_key(packets, window_size):
    # 初始化 defaultdict 用于存储每个 key 的时间戳
    temp_sequence = defaultdict(list)
    p_bits = {}

    # 按 key 收集时间戳
    for key, bits, timestamp, _ in packets:
        temp_sequence[key].append(timestamp)
        p_bits[key] = bits

    # 初始化输出 defaultdict
    rawsequence = defaultdict(list)

    # 对每个 key 单独处理
    for key, timestamps in temp_sequence.items():
        if not timestamps:
            continue

        # 获取该 key 的时间范围
        min_time = min(timestamps)
        max_time = max(timestamps)

        # 计算该 key 的窗口数量
        num_windows = int((max_time - min_time) // window_size) + 1

        # 初始化计数数组
        counters = [0] * num_windows

        # 统计包数
        for time in timestamps:
            window_index = int((time - min_time) // window_size)
            if 0 <= window_index < num_windows:
                counters[window_index] += p_bits[key]

        # 存储到 rawsequence
        rawsequence[key] = counters

    return rawsequence

In [21]:

DATA = './Branch/Datasets/MAWI_input_t.csv'
windowSize = 100_000
queue_length = 8
flowNum = 1387705

'''
Raw packets
'''
packets = []
with open(DATA, 'r') as f:
    next(f)
    for line in f:
        key, p_bits, global_time, label = line.strip().split(',')

        key = int(key)
        global_time = int(global_time)
        label = int(label)
        p_bits = int(p_bits)
        packets.append((key, p_bits, global_time, label))

'''
Perfect traffic statistics method
'''

from collections import defaultdict
interval = 100_000


def packets_to_flow(flowNum, packets, interval):
    timeNow = 0

    replay = [[] for x in range(flowNum)]
    stStorage = defaultdict(int)

    for p in packets:
        flowKey, pBits, pTimeStamp, label = p
        if pTimeStamp - timeNow >= interval:
            for key, val in stStorage.items():
                replay[key].append((timeNow, val))

            stStorage = defaultdict(int)
            timeNow = (pTimeStamp // interval) * interval

        stStorage[flowKey] += pBits

    '''
    pop out all data at the end
    '''
    for key, val in stStorage.items():
        replay[key].append((timeNow, val))

    return replay

no_error_replay = packets_to_flow(flowNum, packets, interval)

final_result = []

for i in [277, 555, 833, 1111, 1388, 1666, 1944, 2222, 2500, 2777]:

    # , 555, 833, 1111, 1388, 1666, 1944, 2222, 2500, 2777

    ts_mon = TsMon(3, i, 1, queue_length, windowSize)


    '''
    运行 TsSketch test
    '''
    for packet in packets:
        key, p_bits, global_time, label = packet
        ts_mon.TsSketch_test(key, p_bits, global_time)


    '''
    精度计算
    '''
    arr = []

    cos = 0
    energy = 0
    are = 0
    count = 0
    for flowID in range(1, flowNum):
        if len(ts_mon.traffic[flowID]) == 0:
            continue
        else:
            count += 1
        _, y_tsmon = zip(*ts_mon.traffic[flowID])
        _, y_raw = zip(*no_error_replay[flowID])
        # 时间序列对齐



        cos += cosine_similarity(y_tsmon, y_raw)
        energy += energy_similarity(y_tsmon, y_raw)
        are += average_relative_error(y_tsmon, y_raw)

        # print(average_relative_error(y_tsmon[:minLen], y_raw[:minLen]), y_tsmon[:minLen], y_raw[:minLen], flowID)

    cos /= count
    energy /= count
    are /= count
    final_result.append((cos, energy, are))
    print(cos, energy, are)

# print(final_result)

AttributeError: 'CountMinSketch' object has no attribute 'TsSketch_test'

In [3]:
from count_min_sketch import CountMinSketch
DATA = './Branch/Datasets/MAWI_input_t.csv'
windowSize = 100_000
queue_length = 8
flowNum = 1387705

'''
Raw packets
'''
packets = []
with open(DATA, 'r') as f:
    next(f)
    for line in f:
        key, p_bits, global_time, label = line.strip().split(',')

        key = int(key)
        global_time = int(global_time)
        label = int(label)
        p_bits = int(p_bits)
        packets.append((key, p_bits, global_time, label))

'''
Perfect traffic statistics method
'''

from collections import defaultdict
interval = 100_000


def packets_to_flow(flowNum, packets, interval):
    timeNow = 0

    replay = [[] for x in range(flowNum)]
    stStorage = defaultdict(int)

    for p in packets:
        flowKey, pBits, pTimeStamp, label = p
        if pTimeStamp - timeNow >= interval:
            for key, val in stStorage.items():
                replay[key].append((timeNow, val))

            stStorage = defaultdict(int)
            timeNow = (pTimeStamp // interval) * interval

        stStorage[flowKey] += pBits

    '''
    pop out all data at the end
    '''
    for key, val in stStorage.items():
        replay[key].append((timeNow, val))

    return replay

no_error_replay = packets_to_flow(flowNum, packets, interval)

final_result = []

for i in [833, 1666, 2500, 3333, 4166, 5000, 5833, 6666, 7500, 8333]:

    # , 555, 833, 1111, 1388, 1666, 1944, 2222, 2500, 2777

    # ts_mon = TsMon(3, i, 1, queue_length, windowSize)

    method = CountMinSketch(3, i, windowSize)


    '''
    运行 TsSketch test
    '''
    for packet in packets:
        key, p_bits, global_time, label = packet
        method.process(key, p_bits, global_time)


    '''
    精度计算
    '''
    arr = []

    cos = 0
    energy = 0
    are = 0
    count = 0
    for flowID in range(1, flowNum):
        if len(method.result[flowID]) == 0:
            continue
        else:
            count += 1
        _, y_tsmon = zip(*method.result[flowID])
        _, y_raw = zip(*no_error_replay[flowID])
        # 时间序列对齐



        cos += cosine_similarity(y_tsmon, y_raw)
        energy += energy_similarity(y_tsmon, y_raw)
        are += average_relative_error(y_tsmon, y_raw)

        # print(average_relative_error(y_tsmon[:minLen], y_raw[:minLen]), y_tsmon[:minLen], y_raw[:minLen], flowID)

    cos /= count
    energy /= count
    are /= count
    final_result.append((cos, energy, are))
    print(cos, energy, are)

# print(final_result)

D:\Anaconda\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Anaconda\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0.9252103493365348 0.9694258804530264 3.8962104193546083
0.9298291046172478 0.9786899188253902 3.895447835163365
0.9303240382594732 0.9798753908472965 3.8948895137426254
0.9303095250248903 0.9796730863274425 3.8950447353947055
0.9304538345278681 0.9800473693924389 3.8948653347668634
0.9303235196091727 0.9798832078731938 3.894876727294721
0.9304538268090011 0.9800476758450516 3.8948625678125683
0.9304538268090011 0.9800476758450516 3.8948625678125683
0.9304538268090011 0.9800476758450516 3.8948625678125683
0.9304538268090011 0.9800476758450516 3.8948625678125683
